<a href="https://colab.research.google.com/github/bjarkedc/RAG_FLAN_T5/blob/main/t5_flan_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rank-bm25 nltk transformers sentencepiece

In [ ]:
import pandas as pd
import re
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
def parse_docs(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    # Regular expression to match <doc> elements
    doc_pattern = re.compile(r'<doc id="([^"]+)" url="([^"]+)" title="([^"]+)">(.*?)</doc>', re.DOTALL)

    # Find all matches
    matches = doc_pattern.findall(file_content)

    # Extract data and create a list of dictionaries
    docs = [{'id': match[0], 'url': match[1], 'title': match[2], 'text': match[3].strip()} for match in matches]
    return docs

# Create a DataFrame
def create_dataframe(docs):
    return pd.DataFrame(docs)

# Preprocess text: tokenize and remove stopwords
def preprocess(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token.isalpha() and token not in stop_words]

file_path = 'drive/MyDrive/apps/data-anlp/starwarsfandomcom-removed-space-and.txt'  # Replace with the actual path to your file
docs = parse_docs(file_path)
df = create_dataframe(docs)
df['tokenized_text'] = df['text'].apply(preprocess)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
df = pd.read_csv('drive/MyDrive/apps/data-anlp/starwarsfandomcom.csv')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

bm25 = BM25Okapi(df['tokenized_text'].tolist())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import pandas as pd
import re
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer
#model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-xxl')

In [ ]:
def query_bm25(query, num_results=10):
    query_tokens = preprocess(query)
    doc_scores = bm25.get_scores(query_tokens)
    top_doc_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:num_results]
    return df.iloc[top_doc_indices]['text'].str.cat(sep=' ')

def generate_response(query, context):
    # Concatenate the query and context
    input_text = f'Answer this question: "{query}". Based off the following context: "{context}".'

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Truncate the input to the model's max length if necessary
    max_length = tokenizer.model_max_length
    if input_ids.size(1) > max_length:
        input_ids = input_ids[:, :max_length]

    truncated_input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)

    output_ids = model.generate(input_ids)[0]

    response = tokenizer.decode(output_ids, skip_special_tokens=True)

    return response, context, truncated_input_text


In [ ]:
text_set_path = '/content/drive/MyDrive/data-anlp/star_wars_dataset_test/'

# Function to read a file and return a list of lines
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file]

# Load the data
questions = read_file_to_list(text_set_path + 'questions.txt')
answers = read_file_to_list(text_set_path + 'answers.txt')

In [ ]:
from itertools import islice
max_pairs = 25

backslash_char = ". \n"
correct_count = 0  # Initialize a counter for correct answers
#{truncated_input_text.replace(". ",backslash_char)}

for query, answer in islice(zip(questions, answers), max_pairs):
    context = query_bm25(query, 15)

    # Generate response
    response, context, truncated_input_text = generate_response(query, context)

    # Check if the response is correct (this might need to be adjusted based on your criteria)
    if response.strip().lower() == answer.strip().lower():
        correct_count += 1

    print(f"""{query}
{response}
{answer}
----------------------------------------------""")

# Print the total count of correct answers after the loop
print(f"Total correct answers: {correct_count}/{max_pairs}")


'May the Fourth be with you' is the official tagline of the galaxy-wide holiday celebrated the first week of May in honor of what sci-fi franchise?
borg
Star Wars
----------------------------------------------
What 2018 Star Wars film focuses on the early adventures of Chewbacca and the character who was portrayed by Harrison Ford in "A New Hope," "The Empire Strikes Back," and "Return of the Jedi?"
'Ndranth Poussan'
Solo
----------------------------------------------
"Rogue One" was a 2016 standalone feature film starring Felicity Jones within the universe (er, galaxy) of what film franchise?
Rebel Alliance
Star Wars
----------------------------------------------
What Admiral in the rebellion, a member of the Mon Calamari race, famously exclaimed 'It's a trap!' during the climactic space battle at the end of 'Return Of The Jedi?'
The X-Men are preparing to launch a new spaceship.
Admiral Ackbar
----------------------------------------------
Sure to mess up Star Wars Day, 63.4% of Texa